In [30]:
import pandas as pd

In [51]:
from get_soccer_data import get_soccer_data

In [ ]:
import logging
import traceback

# Configure the error logger 
logging.basicConfig(
    filename='error_log.txt',
    level=logging.ERROR,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

def log_exception(e: Exception):
    """Logs an exception with full traceback to error_log.txt"""
    logging.error("Exception occurred: \n%s", traceback.format_exc())


In [ ]:
import pandas as pd

def prepare_df(df: pd.DataFrame, column:str = None, new_column:str = None)-> pd.DataFrame:
    """
    Prepare dataframe to be ingested in postgresql,replace table name reserved to database, 
    Change empty strings to 'None' values, replaces all 'NaN' values in the DataFrame with 'None',
    column names to lower case.
    
    Parameters:
    - df: DataFrame to prepare 
    - column: Dataframe column name to be renamed (optional)
    - new_column: New column name (optional)
    """
    if column is not None and new_column is not None:
        df.rename(columns={column:new_column}, inplace= True)
    
    df = df.replace(r'^\s*$', None, regex=True)
    df = df.where(pd.notnull(df), None)
    df.columns = df.columns.str.lower()
    return df

In [ ]:
## Create connection to sql, loading enviromental variables

import psycopg2
from dotenv import load_dotenv
import os

load_dotenv(override=True) 
host = os.getenv("DB_HOST")
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
database = os.getenv("DB_NAME")

pgconn = psycopg2.connect(
    host= host,
    user = user,
    password= password,
    database = database
)


pgcursor = pgconn.cursor()

from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
pgconn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

pgcursor.execute('SELECT current_database()') # Test the connection
pgcursor.fetchone()





('soccerdatawarehouse',)

In [ ]:
from sqlalchemy import create_engine


def df_to_posgres (user:str, host:str, password:str, database:str, df:pd.DataFrame, db_table:str,schema:str):
    """
    Saves DataFrame in database connected 
    
    Parameters:
    - user: posgresql database connection user
    - host: posgresql database connection host
    - password: posgresql database connection host password
    - database: posgresql database name to connect
    - df: DataFrame to save in database
    - db_table: database table in posgresql
    - schema: schema where the db_table is located
    """

    engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}/{database}?options=-csearch_path%3D{schema}')

    df.to_sql(db_table, engine, if_exists='append',index=False)

In [ ]:
## Create staging schema in soccerwarehouse database
schema_name = "staging"
pgcursor.execute(f"CREATE SCHEMA IF NOT EXISTS {schema_name};")

# Commit the transaction
pgconn.commit()

In [ ]:
## Create Table schedule in staging schema 
pgcursor.execute("""
 CREATE TABLE IF NOT EXISTS staging.schedule 
(                 
   GameId                   INT
  , RoundId                 INT
  , Season                  INT
  , SeasonType              INT
  , Group_num               INT
  , AwayTeamId              INT
  , HomeTeamId              INT
  , VenueId                 INT
  , Day                     DATE
  , DateTime                TIMESTAMP
  , Status                  VARCHAR(50)
  , Week                    INT
  , Winner                  VARCHAR(50)
  , VenueType               VARCHAR(50)
  , AwayTeamKey             VARCHAR(50)
  , AwayTeamName            VARCHAR(50)
  , AwayTeamCountryCode     VARCHAR(50)
  , AwayTeamScore           INT
  , AwayTeamScorePeriod1    INT
  , AwayTeamScorePeriod2    INT
  , AwayTeamScoreExtraTime  INT
  , AwayTeamScorePenalty    INT
  , HomeTeamKey             VARCHAR(50)
  , HomeTeamName            VARCHAR(50)
  , HomeTeamCountryCode     VARCHAR(50)
  , HomeTeamScore           INT
  , HomeTeamScorePeriod1    INT
  , HomeTeamScorePeriod2    INT
  , HomeTeamScoreExtraTime  INT
  , HomeTeamScorePenalty    INT
  , Updated                 TIMESTAMP
  , UpdatedUtc              TIMESTAMP
  , GlobalGameId            INT
  , GlobalAwayTeamId        INT
  , GlobalHomeTeamId        INT
  , IsClosed                BOOLEAN
  , PlayoffAggregateScore   INT
);
""")

In [ ]:
## Create Table games in staging schema 
pgcursor.execute("""
 CREATE TABLE IF NOT EXISTS staging.games 
(                 
    GameId                    INT
  , RoundId                   INT
  , Season                    INT
  , SeasonType                INT
  , Group_num                 INT
  , AwayTeamId                INT
  , HomeTeamId                INT
  , VenueId                   INT
  , Day                       TIMESTAMP
  , DateTime                  TIMESTAMP
  , Status                    VARCHAR(255)
  , Week                      INT
  , Period                    VARCHAR(255)
  , Clock                     TIMESTAMP
  , Winner                    VARCHAR(255)
  , VenueType                 VARCHAR(255)
  , AwayTeamKey               VARCHAR(255)
  , AwayTeamName              VARCHAR(255)
  , AwayTeamCountryCode       VARCHAR(255)
  , AwayTeamScore             INT
  , AwayTeamScorePeriod1      INT
  , AwayTeamScorePeriod2      INT
  , AwayTeamScoreExtraTime    TIMESTAMP
  , AwayTeamScorePenalty      TIMESTAMP
  , HomeTeamKey               VARCHAR(255)
  , HomeTeamName              VARCHAR(255)
  , HomeTeamCountryCode       VARCHAR(255)
  , HomeTeamScore             INT
  , HomeTeamScorePeriod1      INT
  , HomeTeamScorePeriod2      INT
  , HomeTeamScoreExtraTime    INT
  , HomeTeamScorePenalty      INT
  , HomeTeamMoneyLine         INT
  , AwayTeamMoneyLine         INT
  , DrawMoneyLine             INT
  , PointSpread               FLOAT
  , HomeTeamPointSpreadPayout INT
  , AwayTeamPointSpreadPayout INT
  , OverUnder                 FLOAT
  , OverPayout                INT
  , UnderPayout               INT
  , Attendance                INT
  , Updated                   TIMESTAMP
  , UpdatedUtc                TIMESTAMP
  , GlobalGameId              INT
  , GlobalAwayTeamId          INT
  , GlobalHomeTeamId          INT
  , ClockExtra                TIMESTAMP
  , ClockDisplay              TIMESTAMP
  , IsClosed                  BOOLEAN
  , HomeTeamFormation         VARCHAR(255)
  , AwayTeamFormation         VARCHAR(255)
  , PlayoffAggregateScore     TIMESTAMP
  , awayteamcoachid           INT
  , hometeamcoachid           INT
  , mainreferee               INT
  , assistantreferee1         INT
  , assistantreferee2         INT
  , fourthreferee             INT
  , videoassistantreferee     INT                
);
""")

In [ ]:
## Create Table referees in staging schema 
pgcursor.execute("""
 CREATE TABLE IF NOT EXISTS staging.referees 
(                 
    RefereeId     INT
  , FirstName     VARCHAR(50)
  , LastName      VARCHAR(50)
  , ShortName     VARCHAR(50)
  , Nationality   VARCHAR(50)
);
""")

In [ ]:
## Create Table coaches in staging schema 
pgcursor.execute("""
 CREATE TABLE IF NOT EXISTS staging.coaches
(                 
    CoachId      INT
  , FirstName    VARCHAR(50)
  , LastName     VARCHAR(50)
  , ShortName    VARCHAR(50)
  , Nationality  VARCHAR(50)
);
""")

In [ ]:
## Create Table goals in staging schema 
pgcursor.execute("""
 CREATE TABLE IF NOT EXISTS staging.goals 
(                 
    GoalId                  INT
  , GameId                  INT
  , TeamId                  INT
  , PlayerId                INT
  , Name                    VARCHAR(50)
  , Type                    VARCHAR(50)
  , AssistedByPlayerId1     INT
  , AssistedByPlayerName1   VARCHAR(50)
  , AssistedByPlayerId2     INT
  , AssistedByPlayerName2   VARCHAR(50)
  , GameMinute              INT
  , GameMinuteExtra         INT
);
""")

In [ ]:
## Create Table lineups in staging schema 
pgcursor.execute("""
 CREATE TABLE IF NOT EXISTS staging.lineups 
(                 
    LineupId                    INT
  , GameId                      INT
  , Type                        VARCHAR(100)
  , TeamId                      INT
  , PlayerId                    VARCHAR(50)
  , Name                        VARCHAR(50)
  , Position                    VARCHAR(50)
  , ReplacedPlayerId            INT
  , ReplacedPlayerName          VARCHAR(50)
  , GameMinute                  VARCHAR(50)
  , GameMinuteExtra             INT
  , PitchPositionHorizontal     FLOAT
  , PitchPositionVertical       FLOAT
);
""")

In [ ]:
## Create Table schedule in bookings schema 
pgcursor.execute("""
 CREATE TABLE IF NOT EXISTS staging.bookings
(                 
    BookingId         INT
  , GameId            INT
  , Type              VARCHAR(100)
  , TeamId            INT
  , PlayerId          VARCHAR(50)
  , Name              VARCHAR(50)
  , GameMinute        INT
  , GameMinuteExtra   VARCHAR(50)
);
""")

In [ ]:
## Create Table schedule in staging teamgames 
pgcursor.execute("""
 CREATE TABLE IF NOT EXISTS staging.teamgames 
(
    StatId                         INT
    , SeasonType                    INT
    , Season                        INT
    , RoundId                       INT
    , TeamId                        INT
    , Name                          VARCHAR(255)
    , Team                          VARCHAR(255)
    , GlobalTeamId                  INT
    , Possession                    FLOAT
    , GameId                        INT
    , OpponentId                    INT
    , Opponent                      VARCHAR(255)
    , Day                           TIMESTAMP
    , DateTime                      TIMESTAMP
    , HomeOrAway                    VARCHAR(255)
    , IsGameOver                    BOOLEAN
    , GlobalGameId                  INT
    , GlobalOpponentId              INT
    , Updated                       TIMESTAMP
    , UpdatedUtc                    TIMESTAMP
    , Games                         INT
    , FantasyPoints                 FLOAT
    , FantasyPointsFanDuel          VARCHAR(50)
    , FantasyPointsDraftKings       FLOAT
    , FantasyPointsYahoo            VARCHAR(50)
    , FantasyPointsMondogoal        FLOAT
    , Minutes                       FLOAT
    , Goals                         FLOAT
    , Assists                       FLOAT
    , Shots                         FLOAT
    , ShotsOnGoal                   FLOAT
    , YellowCards                   FLOAT
    , RedCards                      FLOAT
    , YellowRedCards                FLOAT
    , Crosses                       FLOAT
    , TacklesWon                    FLOAT
    , Interceptions                 FLOAT
    , OwnGoals                      FLOAT
    , Fouls                         FLOAT
    , Fouled                        FLOAT
    , Offsides                      FLOAT
    , Passes                        FLOAT
    , PassesCompleted               FLOAT
    , LastManTackle                 FLOAT
    , CornersWon                    FLOAT
    , BlockedShots                  FLOAT
    , Touches                       FLOAT
    , DefenderCleanSheets           FLOAT
    , GoalkeeperSaves               FLOAT
    , GoalkeeperGoalsAgainst        FLOAT
    , GoalkeeperSingleGoalAgainst   FLOAT
    , GoalkeeperCleanSheets         FLOAT
    , GoalkeeperWins                FLOAT
    , PenaltyKickGoals              FLOAT
    , PenaltyKickMisses             FLOAT
    , PenaltyKickSaves              FLOAT
    , PenaltiesWon                  FLOAT
    , PenaltiesConceded             FLOAT
    , Score                         FLOAT
    , OpponentScore                 FLOAT
    , Tackles                       FLOAT
);
""")

In [ ]:
## Create Table schedule in staging playerteams 
pgcursor.execute("""
 CREATE TABLE IF NOT EXISTS staging.playerteams 
( 
     StatId                         INT
    , SeasonType                    INT
    , Season                        INT
    , RoundId                       INT
    , TeamId                        INT
    , PlayerId                      INT
    , Name                          VARCHAR(150)
    , ShortName                     VARCHAR(50)
    , Team                          VARCHAR(255)
    , PositionCategory              VARCHAR(10)
    , Position                      VARCHAR(10)
    , Jersey                        INT
    , Started                       INT
    , Captain                       BOOLEAN
    , Suspension                    BOOLEAN
    , SuspensionReason              VARCHAR(255)
    , FanDuelSalary                 VARCHAR(255)
    , DraftKingsSalary              VARCHAR(255)
    , YahooSalary                   VARCHAR(255)
    , MondogoalSalary               VARCHAR(255)
    , FanDuelPosition               VARCHAR(255)
    , DraftKingsPosition            VARCHAR(255)
    , YahooPosition                 VARCHAR(255)
    , MondogoalPosition             VARCHAR(255)
    , InjuryStatus                  VARCHAR(255)
    , InjuryBodyPart                VARCHAR(255)
    , InjuryNotes                   VARCHAR(255)
    , InjuryStartDate               TIMESTAMP
    , GlobalTeamId                  INT
    , GameId                        INT
    , OpponentId                    INT
    , Opponent                      VARCHAR(255)
    , Day                           TIMESTAMP
    , DateTime                      TIMESTAMP
    , HomeOrAway                    VARCHAR(255)
    , IsGameOver                    BOOLEAN
    , GlobalGameId                  INT
    , GlobalOpponentId              INT
    , Updated                       TIMESTAMP
    , UpdatedUtc                    TIMESTAMP
    , Games                         INT
    , FantasyPoints                 FLOAT
    , FantasyPointsFanDuel          VARCHAR(255)
    , FantasyPointsDraftKings       FLOAT
    , FantasyPointsYahoo            VARCHAR(255)
    , FantasyPointsMondogoal        FLOAT
    , Minutes                       FLOAT
    , Goals                         FLOAT
    , Assists                       FLOAT
    , Shots                         FLOAT
    , ShotsOnGoal                   FLOAT
    , YellowCards                   FLOAT
    , RedCards                      FLOAT
    , YellowRedCards                FLOAT
    , Crosses                       FLOAT
    , TacklesWon                    FLOAT
    , Interceptions                 FLOAT
    , OwnGoals                      FLOAT
    , Fouls                         FLOAT
    , Fouled                        FLOAT
    , Offsides                      FLOAT
    , Passes                        FLOAT
    , PassesCompleted               FLOAT
    , LastManTackle                 FLOAT
    , CornersWon                    FLOAT
    , BlockedShots                  FLOAT
    , Touches                       FLOAT
    , DefenderCleanSheets           FLOAT
    , GoalkeeperSaves               FLOAT
    , GoalkeeperGoalsAgainst        FLOAT
    , GoalkeeperSingleGoalAgainst   FLOAT
    , GoalkeeperCleanSheets         FLOAT
    , GoalkeeperWins                FLOAT
    , PenaltyKickGoals              FLOAT
    , PenaltyKickMisses             FLOAT
    , PenaltyKickSaves              FLOAT
    , PenaltiesWon                  FLOAT
    , PenaltiesConceded             FLOAT
    , Score                         FLOAT
    , OpponentScore                 FLOAT
    , Tackles                       FLOAT
);
""")

In [ ]:
## Fetch and save in database game schedules for the 2025 spanish season
df = get_soccer_data('scores','SchedulesBasic','ESP','2025')
df = prepare_df(df,'Group','Group_num')
df_to_posgres (user, host, password, database, df, 'schedule','staging')

https://api.sportsdata.io/v4/soccer/scores/json/SchedulesBasic/ESP/2025
Status Code: 200
✅ Saved: SchedulesBasic_ESP_2025.json and SchedulesBasic_ESP_2025.csv


In [ ]:
## Get gameid for all games in season 
pgcursor.execute('SELECT DISTINCT gameid from staging.schedule')
gameid_2025_esp = pgcursor.fetchall()
gameid_2025_esp = [str(x[0]) for x in gameid_2025_esp] # Cast gameid to string

In [ ]:
schema = 'staging'  
gameid_errors = []  # list to record gameid with errors 
records = len(gameid_2025_esp) # total of records 
for i in gameid_2025_esp[0:]: # for all gamesid (all games from season 2025)
    try:
        df = get_soccer_data('Stats','BoxScoreFinal','ESP',i) #Fetch data from gameid
        Game_df = prepare_df(pd.json_normalize(df['Game'][0]), 'Group','Group_num') # save game data to Dataframe
        AwayTeamCoach_df = prepare_df(pd.json_normalize(df['AwayTeamCoach'])) 
        HomeTeamCoach_df = prepare_df(pd.json_normalize(df['HomeTeamCoach'])) 
        MainReferee_df = prepare_df(pd.json_normalize(df['MainReferee'])) 
        AssistantReferee1_df = prepare_df(pd.json_normalize(df['AssistantReferee1'])) 
        AssistantReferee2_df = prepare_df(pd.json_normalize(df['AssistantReferee2'])) 
        FourthReferee_df = prepare_df(pd.json_normalize(df['FourthReferee'])) 
        VideoAssistantReferee_df = prepare_df(pd.json_normalize(df['VideoAssistantReferee'])) 
        Lineups_df = prepare_df(pd.json_normalize(df['Lineups'][0])) 
        Goals_df = pd.json_normalize(df['Goals'][0]) 
        Bookings_df = pd.json_normalize(df['Bookings'][0]) 
        TeamGames_df = prepare_df(pd.json_normalize(df['TeamGames'][0]))
        PlayerGames_df = prepare_df(pd.json_normalize(df['PlayerGames'][0]))
        Game_df.loc[0, 'awayteamcoachid'] = AwayTeamCoach_df['coachid'][0] # Add to Game_df relationships for coachs and referees
        Game_df.loc[0, 'hometeamcoachid'] = HomeTeamCoach_df['coachid'][0]
        Game_df.loc[0, 'mainreferee'] = MainReferee_df['refereeid'][0]
        Game_df.loc[0, 'assistantreferee1'] = AssistantReferee1_df['refereeid'][0]
        Game_df.loc[0, 'assistantreferee2'] = AssistantReferee2_df['refereeid'][0]
        Game_df.loc[0, 'fourthreferee'] = FourthReferee_df['refereeid'][0]
        Game_df.loc[0, 'videoassistantreferee'] = VideoAssistantReferee_df['refereeid'][0]
        df_to_posgres(user, host, password, database, AwayTeamCoach_df,'coaches',schema) # Save in database
        df_to_posgres(user, host, password, database, HomeTeamCoach_df,'coaches',schema)
        df_to_posgres(user, host, password, database, MainReferee_df,'referees',schema)
        df_to_posgres(user, host, password, database, AssistantReferee1_df,'referees',schema)
        df_to_posgres(user, host, password, database, AssistantReferee2_df,'referees',schema)
        df_to_posgres(user, host, password, database, FourthReferee_df,'referees',schema)
        df_to_posgres(user, host, password, database, VideoAssistantReferee_df,'referees',schema)
        df_to_posgres(user, host, password, database, Lineups_df,'lineups',schema)
        if len(Goals_df.columns) > 0: # Check if the game had goals
            Goals_df = prepare_df(Goals_df)
            df_to_posgres(user, host, password, database, Goals_df,'goals',schema)
        if len(Bookings_df.columns) > 0: # Check if the game had bookings
            Bookings_df = prepare_df(Bookings_df)
            df_to_posgres(user, host, password, database, Bookings_df,'bookings',schema)
        df_to_posgres(user, host, password, database, TeamGames_df,'teamgames',schema)
        df_to_posgres(user, host, password, database, PlayerGames_df,'playerteams',schema)
        df_to_posgres(user, host, password, database, Game_df,'games',schema)
        print(f'{gameid_2025_esp.index(i)+1}/{records} processed') # print general progress
    except Exception as e:
        print(f"An error occurred: {e}, when saving record {i}") # print error
        gameid_errors.append(i) # record gameid with error
        log_exception(e) # save error to file
        print(f'{gameid_2025_esp.index(i)+1}/{records} processed') # print general progress
    

https://api.sportsdata.io/v4/soccer/Stats/json/BoxScoreFinal/ESP/79260
Status Code: 200
✅ Saved: BoxScoreFinal_ESP_79260.json and BoxScoreFinal_ESP_79260.csv
1/380 processed
https://api.sportsdata.io/v4/soccer/Stats/json/BoxScoreFinal/ESP/79433
Status Code: 200
✅ Saved: BoxScoreFinal_ESP_79433.json and BoxScoreFinal_ESP_79433.csv
2/380 processed
https://api.sportsdata.io/v4/soccer/Stats/json/BoxScoreFinal/ESP/79431
Status Code: 200
✅ Saved: BoxScoreFinal_ESP_79431.json and BoxScoreFinal_ESP_79431.csv
3/380 processed
https://api.sportsdata.io/v4/soccer/Stats/json/BoxScoreFinal/ESP/79326
Status Code: 200
✅ Saved: BoxScoreFinal_ESP_79326.json and BoxScoreFinal_ESP_79326.csv
4/380 processed
https://api.sportsdata.io/v4/soccer/Stats/json/BoxScoreFinal/ESP/79258
Status Code: 200
✅ Saved: BoxScoreFinal_ESP_79258.json and BoxScoreFinal_ESP_79258.csv
5/380 processed
https://api.sportsdata.io/v4/soccer/Stats/json/BoxScoreFinal/ESP/79467
Status Code: 200
✅ Saved: BoxScoreFinal_ESP_79467.json and 

In [ ]:
print(gameid_errors) # check if errors occurred

[]
